<a href="https://colab.research.google.com/github/LukasSales/Desafio_back_end_2/blob/feature%2Fcarregamento-e-limpeza/Desafio_back_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Desafio_2

##Bibliotecas Necessárias

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import os
import xgboost as xgb

!pip install --upgrade scikit-learn
!python -m textblob.download_corpora

from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)

# Configurar visualizações
%matplotlib inline
sns.set(style="whitegrid")

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


##Carregar o Dataset

In [3]:
# Download latest version
path = kagglehub.dataset_download("pypiahmad/goodreads-book-reviews1")

print("Path to dataset files:", path)

100%|██████████| 8.14G/8.14G [03:43<00:00, 39.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1


In [4]:
print("Caminho para os arquivos do dataset:", path)

# Listar todos os arquivos e diretórios
for root, dirs, files in os.walk(path):
    for filename in files:
        print(os.path.join(root, filename))

Caminho para os arquivos do dataset: /root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1
/root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1/book_clubs.json
/root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1/goodreads_books.json
/root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1/goodreads_book_works.json
/root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1/goodreads_book_series.json
/root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1/download.ipynb
/root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1/goodreads_reviews_dedup.json
/root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1/goodreads_interactions.csv
/root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1/reviews.ipynb
/root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1/goodreads_book_gen

###Carregar o Dataset no DataFrame

In [5]:
n = 100000  # Número de linhas a serem lidas

In [6]:
json_file = os.path.join(path, 'goodreads_reviews_dedup.json')

# Carregar uma amostra do arquivo para evitar sobrecarregar a memória
df = pd.read_json(json_file, lines=True, chunksize=n)
df = next(df)

In [7]:
json_file_books = os.path.join(path, 'goodreads_books.json')

# Carregar o arquivo
df_books = pd.read_json(json_file_books, lines=True, chunksize=n)
df_books = next(df_books)

In [8]:
json_file_authors = os.path.join(path, 'goodreads_book_authors.json')

# Carregar o arquivo
df_authors = pd.read_json(json_file_authors, lines=True, chunksize=n)
df_authors = next(df_authors)

In [9]:
print(df_books.columns)

Index(['isbn', 'text_reviews_count', 'series', 'country_code', 'language_code',
       'popular_shelves', 'asin', 'is_ebook', 'average_rating', 'kindle_asin',
       'similar_books', 'description', 'format', 'link', 'authors',
       'publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'edition_information', 'publication_year', 'url',
       'image_url', 'book_id', 'ratings_count', 'work_id', 'title',
       'title_without_series'],
      dtype='object')


In [10]:
print("Estrutura do DataFrame de Autores:")
print(df_authors.head())
print(df_authors.columns)

Estrutura do DataFrame de Autores:
   average_rating  author_id  text_reviews_count              name  \
0            3.98     604031                   7  Ronald J. Fields   
1            4.08     626222               28716     Anita Diamant   
2            3.92      10333                5075    Barbara Hambly   
3            3.68       9212               36262   Jennifer Weiner   
4            3.82     149918                  96     Nigel Pennick   

   ratings_count  
0             49  
1         546796  
2         122118  
3         888522  
4           1740  
Index(['average_rating', 'author_id', 'text_reviews_count', 'name',
       'ratings_count'],
      dtype='object')


In [11]:
df_authors['author_id'] = df_authors['author_id'].astype(str)

In [12]:
#verificar os tipos antes do merge
print(df['book_id'].dtype)
print(df_books['book_id'].dtype)

int64
int64


In [13]:
# Selecionar apenas as colunas necessárias do DataFrame de livros
columns_books = ['book_id', 'title', 'authors', 'popular_shelves']
df_books_selected = df_books[columns_books]

# Realizar o merge interno para evitar NaNs
df = pd.merge(
    df,
    df_books,
    on='book_id',
    how='inner'
)
print(f"Total de linhas após o merge: {df.shape[0]}")

Total de linhas após o merge: 4549


####Explorar o DataFrame

In [14]:
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4549 entries, 0 to 4548
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   user_id               4549 non-null   object
 1   book_id               4549 non-null   int64 
 2   review_id             4549 non-null   object
 3   rating                4549 non-null   int64 
 4   review_text           4549 non-null   object
 5   date_added            4549 non-null   object
 6   date_updated          4549 non-null   object
 7   read_at               4549 non-null   object
 8   started_at            4549 non-null   object
 9   n_votes               4549 non-null   int64 
 10  n_comments            4549 non-null   int64 
 11  isbn                  4549 non-null   object
 12  text_reviews_count    4549 non-null   object
 13  series                4549 non-null   object
 14  country_code          4549 non-null   object
 15  language_code         4549 non-null   

,book_id,rating,n_votes,n_comments
count,4.549000e+03,4549.000000,4549.000000,4549.000000
mean,1.187329e+07,3.699275,0.917344,0.204001
std,1.005823e+07,1.236413,5.324867,1.055693
min,4.200000e+02,0.000000,0.000000,0.000000
25%,9.725280e+05,3.000000,0.000000,0.000000
50%,1.119362e+07,4.000000,0.000000,0.000000
75%,1.877436e+07,5.000000,1.000000,0.000000
max,3.623267e+07,5.000000,280.000000,43.000000


In [15]:
df['average_rating'] = pd.to_numeric(df['average_rating'], errors='coerce')
df['ratings_count'] = pd.to_numeric(df['ratings_count'], errors='coerce')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4549 entries, 0 to 4548
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user_id               4549 non-null   object 
 1   book_id               4549 non-null   int64  
 2   review_id             4549 non-null   object 
 3   rating                4549 non-null   int64  
 4   review_text           4549 non-null   object 
 5   date_added            4549 non-null   object 
 6   date_updated          4549 non-null   object 
 7   read_at               4549 non-null   object 
 8   started_at            4549 non-null   object 
 9   n_votes               4549 non-null   int64  
 10  n_comments            4549 non-null   int64  
 11  isbn                  4549 non-null   object 
 12  text_reviews_count    4549 non-null   object 
 13  series                4549 non-null   object 
 14  country_code          4549 non-null   object 
 15  language_code        

###Valores Nulos

In [51]:
# Análise de valores nulos
print("\nNúmero de valores nulos após o tratamento:")
print(df.isnull().sum())


Número de valores nulos após o tratamento:
user_id                 0
book_id                 0
review_id               0
rating                  0
review_text             0
date_added              0
date_updated            0
read_at                 0
started_at              0
n_votes                 0
n_comments              0
isbn                    0
text_reviews_count      0
series                  0
country_code            0
language_code           0
popular_shelves         0
asin                    0
is_ebook                0
average_rating          0
kindle_asin             0
similar_books           0
description             0
format                  0
link                    0
authors                 0
publisher               0
num_pages               0
publication_day         0
isbn13                  0
publication_month       0
edition_information     0
publication_year        0
url                     0
image_url               0
ratings_count           0
work_id             

In [54]:
df['average_rating'] = df['average_rating'].fillna(df['average_rating'].mean())
df['ratings_count'] = df['ratings_count'].fillna(0)
df['text_reviews_count'] = df['text_reviews_count'].fillna(0)
df['popular_shelves'] = df['popular_shelves'].fillna('[]')
df['authors'] = df['authors'].fillna('[]')

In [57]:
# Verificar e remover duplicatas
num_duplicatas = df.duplicated(subset=['user_id', 'book_id']).sum()
print(f"Número de registros duplicados (baseado em user_id e book_id): {num_duplicatas}")

Número de registros duplicados (baseado em user_id e book_id): 0


In [58]:
if num_duplicatas > 0:
    df = df.drop_duplicates(subset=['user_id', 'book_id'])
    print("Duplicatas removidas.")
else:
    print("Nenhuma duplicata encontrada com base em user_id e book_id.")

Nenhuma duplicata encontrada com base em user_id e book_id.
